# PyGMT - A Python interface for the Generic Mapping Tools

## Why use PyGMT?

To get access to GMT's functionality directly from Python!


## Project goals

- Make GMT more accessible to new users.
- Build a Pythonic API for GMT.
- Interface with the GMT C API directly using ctypes (no system calls).
- Support for rich display in the Jupyter notebook.
- Integration with the PyData ecosystem: numpy ndarray or pandas DataFrame for data tables, xarray DataArray for grids and geopandas GeoDataFrame for geographical data.

## Where to find out more

Take a look at our [Tutorials](https://www.pygmt.org/latest/tutorials),
visit the [PyGMT Gallery](https://www.pygmt.org/latest/gallery), and
some [external PyGMT examples](https://www.pygmt.org/latest/external_resources.html)!

### About this demo

In this demo, we will go over the same examples from earlier in this short course but in Python! The workflow for each example would likely be different if starting in Python, but hopefully this will be a helpful learning exercise for the differences between CLI GMT and PyGMT.

## Basics

In [ ]:
import pygmt
import numpy as np
import pandas as pd

In [ ]:
# Load the class first. This only needs to be run once
from IPython.display import Image

### Bash example

In [ ]:
%%bash
# GMT version
gmt begin italy png
    gmt coast -R5/20/35/50 -Wthin -Gwheat -EIT+gred -Df -Sazure -B -N1/thick,red -JM15c
    gmt inset begin -DjTR+w4c+o0.2c -F+gwhite+pthick
        gmt coast -Rg -JG10E/25N/? -Gwheat -Bg -EIT+gred
    gmt inset end
gmt end

In [ ]:
Image('italy.png', width=500)

### Python example

In [ ]:
# Create an instance of the figure class
fig = pygmt.Figure()
# Plot coastlines, land, water, country borders, and a map frame
fig.coast(
    region=[5,20,35,50],
    shorelines="thin",
    land="wheat",
    dcw="IT+gred",
    water="azure",
    frame=True,
    borders="1/thick,red",
    projection="M15c"
)
# Plot an inset with global map
with fig.inset(position="jTR+w4c+o0.2c", box="+gwhite+pthick"):
    fig.coast(region="g", projection="G10E/25N/?", land="wheat", frame="g", dcw="IT+gred")
# Display the result
fig.show()

## Lines and Symbols

### Bash example

In [ ]:
%%bash
gmt begin lines png
    # Continous red line
    echo  0 0  > t.dat
    echo 10 1 >> t.dat
    gmt plot t.dat -R-1/11/-1/11 -JX14 -Baf -BWSen -W1,red

    # Continous thick green line
    echo  0 1  > t.dat
    echo 10 2 >> t.dat
    gmt plot t.dat -W3,green

    # Dashed thick blue line
    echo  0 2  > t.dat
    echo 10 3 >> t.dat
    gmt plot t.dat -W3,blue,dashed

    # Dotted black line
    echo  0 3  > t.dat
    echo 10 4 >> t.dat
    gmt plot t.dat -W1,black,dotted

    # Dash-Dot green line
    echo  0 4  > t.dat
    echo 10 5 >> t.dat
    gmt plot t.dat -W1,0/150/0,-.

    # Using round corners
    echo  0 5  > t.dat
    echo 10 6 >> t.dat
    gmt plot t.dat --PS_LINE_CAP=round -W4,black,dashed

    # Annotated line
    echo  0 7  > t.dat
    echo 10 7 >> t.dat
    gmt plot t.dat  -Sqn15:+f25+l~ -W1,sienna

    echo 5 7.1 Now, a bit harder | gmt text -C -F+f18+jMC -Gwhite    
    # Simulating circles with a dashed line
    echo  0 8  > t.dat
    echo 10 9 >> t.dat
    gmt plot t.dat --PS_LINE_CAP=round -W10,orange,0_20:0

    # Alternating larger and smaller circles
    echo  0  9  > t.dat
    echo 10 10 >> t.dat
    gmt plot t.dat --PS_LINE_CAP=round -W6,brown,0_20:0

    echo  0  9  > t.dat
    echo 10 10 >> t.dat
    gmt plot t.dat --PS_LINE_CAP=round -W3,green,0_20:10
gmt end


In [ ]:
Image('lines.png', width=500)

### Python example

In [ ]:
x = np.array([0, 10])
y = np.array([0, 1])


fig = pygmt.Figure()
fig.plot(x=x, y=y, region=[-1, 11,  -1, 11], projection="X14", frame=["af", "WSen"], pen="1,red")
fig.plot(x=x, y=y+1, pen="3,green")
fig.plot(x=x, y=y+2, pen="3,blue,dashed")
fig.plot(x=x, y=y+3, pen="1,black,dotted")
fig.plot(x=x, y=y+4, pen="1,0/150/0,-.")
with pygmt.config(PS_LINE_CAP="round"):
    fig.plot(x=x, y=y+5, pen="4,black,dashed")
fig.plot(x=x, y=y+6, pen="1,sienna", style="qn15:+f25+l~")
with pygmt.config(PS_LINE_CAP="round"):
    fig.plot(x=x, y=y+7, pen="10,orange,0_20:0")
    fig.plot(x=x, y=y+8, pen="6,brown,0_20:0")
    fig.plot(x=x, y=y+8, pen="3,green,0_20:10")
fig.show()

## Grids

### Bash example

In [ ]:
%%bash
gmt begin 5_exercise png

    # Set a Mercator projection, and region for Argentina.
    gmt basemap -RAR,FK,GS+r2 -JM15c -B+n
    
    # Set the default freme (-Baf) and add a title to the plot (-B+t).
    gmt basemap -Baf -B+t"Relief map of Argentina"

    # Plot the earth synbath data using the "oleron" CPT. Adding shading at a 45
    # degree azimuth (+a45) with intensity scaled to 2 (+nt2)
    gmt grdimage @earth_synbath -I+a45+nt2 -Coleron

    # Plot contours on top of the shaded grid. Limit to negative contours only
    # (-Ln) and ommit contours less 1000 km long (-Q). For the annotations,
    # configure the font size to be 6pt (+f).
    gmt grdcontour @earth_synbath -Ln -Q1000k \
        -C200 -Wcthinnest,gray20 \
        -A1000+f6p+gwhite+p -Wathinnest,gray20

    # Plot the country borders (-N1) in white.
    gmt coast -N1/white

    # Plate a colorbar inside ner the right top with and offset in x (+o) and
    # customize the width and height (+w). Set the label interval (-B) and add
    # an annotation to the x axis (-Bx+l). Add a frame around the colorbar (-F).
    gmt colorbar -DjTR+o1.7c/0.4c+w70%/0.5c -I -Ba1 -Bx+l"km" -W0.001 -F+gwhite+p+r+s

gmt end

In [ ]:
Image('5_exercise.png', width=500)

### Python example

In [ ]:
fig = pygmt.Figure()
fig.basemap(region="AR,FK,GS+r2", projection="M15c", frame="+n")
fig.basemap(frame=["af","+tRelief map of Argentina"])
fig.grdimage("@earth_synbath", shading="+a45+nt2", cmap="oleron")
fig.grdcontour(
    "@earth_synbath",
    limit="n",
    cut="1000k",
    interval=200,
    pen=["cthinnest,gray20", "athinnest,gray20"],
    annotation="1000+f6p+gwhite+p"
)
fig.coast(borders="1/white")
fig.colorbar(
    position="jTR+o1.7c/0.4c+w70%/0.5c",
    shading=True,
    frame=["a1", "x+lkm"],
    scale=0.001,
    box="+gwhite+p+r+s"
)
fig.show()

## Seismology

### Bash example

In [ ]:
%%bash
gmt begin
    # Figure 1: Beachballs on maps
    gmt figure japan-beachballs-mapview png

    # plot basemap, setting map width, region and frames
    gmt basemap -R122/147/30/48 -JM15c -BWSen -Baf
    # Plot Earth relief by mapping values to colors using the "globe" CPT
    # Set automatic shading effects
    gmt grdimage @earth_relief_02m -I+d
    # Add the colorbar to the Top Right (TR) and shift to the right by 2 cm.
    # The colorbar width is set to be 6 cm.
    gmt colorbar -DjTR+o-2c/0c+w6c+ml -Ba2000+l"Earth relief (m)"

    # Customize the jet CPT for earthquake depth from 0 to 700
    gmt makecpt -Cjet -T0/700/100
    # -C: Color the beachballs based on earthquake depths and the "current" CPT
    gmt meca ../5_seismology/japan-focal.dat -Sd0.5c+f0p -C
    # Add the colorbar to the Bottom Right (BR) and shift to the right by 2 cm.
    # The colorbar is reversed by giving a negative length (+w-6c).
    gmt colorbar -DjBR+o-2c/0c+w-6c+ml -Ba100+l"Focal depth (km)"

    # Prepare data file for plotting and label the cross-section line
    echo 126 42 A > xsection.dat
    echo 146 40 B >> xsection.dat
    # plot the cross-section line, with a 2 point, red pen
    # the plot column only needs the longitude and latitude of the input file, and ignores the third column
    gmt plot xsection.dat -W2p,red
    # label the cross-section by "A" and "B", with a font size of 15 points
    gmt text xsection.dat -F+f15p


    # Figure 2: Beachball cross-sections
    gmt figure japan-beachballs-sideview png
    # Customize the jet CPT for earthquake depth from 0 -700
    gmt makecpt -Cjet -T0/700/100
    # The cross-section is selected by specifly the locations (longitude and latitude) of a starting point (126/42),
    # and a ending point (146/40). The cross-section plane is vertical (dip angle=90), with the width set to be 500 km,
    # and depth to be 0-70 km.
    #
    # To reverse the Y axis, set the figure heigth to a negative value (-10 cm)
    gmt coupe ../5_seismology/japan-focal.dat -Sd0.5c+f0p -Aa126/42/146/40/700+r+w500 -Q -C \
        -JX15c/-10c -BWSen -Bxaf+l"Distance (km)" -Byaf+l"Depth (km)"
    # Add the colorbar to the Bottom Right (BR) and shift to the right by 2 cm.
    # The colorbar is reversed by giving a negative length (+w-10c).
    gmt colorbar -DjBR+o-2c/0c+w-10c+ml -Np -Ba100+l"Focal depth (km)"

    # cleanup temporary files
    rm xsection.dat
gmt end


In [ ]:
Image('japan-beachballs-mapview.png', width=500)

### Python example

In [ ]:
Image('japan-beachballs-sideview.png', width=500)

In [ ]:
xcross = np.array([126, 146])
ycross = np.array([42, 40])
text = np.array(["A", "B"])
fig = pygmt.Figure()
fig.basemap(region=[122,147,30,48], projection="M15c", frame=["WSen", "af"])
fig.grdimage("@earth_relief_02m", shading="+a")
fig.colorbar(position="jTR+o-2c/0c+w6c+ml", frame='a2000+l"Earth relief (m)"')
pygmt.makecpt(cmap="jet", series=[0, 700, 100])
fig.meca('../5_seismology/japan-focal.dat', convention="mt", scale="0.5c+f0p", C=True)
fig.colorbar(position="jBR+o-2c/0c+w-6c+ml", frame='a100+l"Focal depth (km)"')
fig.plot(x=xcross, y=ycross, pen="2p,red")
fig.text(x=xcross, y=ycross, text=text, font="15p")
fig.show()

### Python example

In [ ]:
fig = pygmt.Figure()
pygmt.makecpt(cmap="jet", series=[0, 700, 100])
with pygmt.clib.Session() as session:
    session.call_module(
        "coupe",
        '../5_seismology/japan-focal.dat -Sd0.5c+f0p -Aa126/42/146/40/700+r+w500 -Q -C -JX15c/-10c -BWSen -Bxaf+l"Distance (km)" -Byaf+l"Depth (km)"'
    )
fig.colorbar(position="jBR+o-2c/0c+w-10c+ml", N="p", frame='a100+l"Focal depth (km)"')
fig.show()

## Geodesy

### Bash example

In [ ]:
%%bash
#!/bin/bash
dem="../6_geodesy/dem_dsamp.grd"
gps="../6_geodesy/GPS.dat"
RR=`gmt grdinfo -I- $dem`
output="demo"
format="png"

gmt begin $output $format
    gmt basemap -JM6 $RR -Ba1f0.5
    gmt grdimage $dem -I+nt.3 -Cgeo
    gmt coast -Na/0.5p,black,-.- -Slightblue -Df
    gmt grdsample ../6_geodesy/dE.grd -I0.1 -Gtmpe.grd
    gmt grdsample ../6_geodesy/dN.grd -I0.1 -Gtmpn.grd
    gmt grd2xyz tmpe.grd > tmpe.lld
    gmt grd2xyz tmpn.grd > tmpn.lld
    paste tmpe.lld tmpn.lld | awk '{print $1,$2,$3,$6,"0","0","0"}' > defo.dat

    gmt velo defo.dat -W0.1p,black -Se0.02/0.65/10 -A10p+eA+n10
    gmt plot ../6_geodesy/new_faults.gmt -W0.5p,red
    gmt velo $gps -W1p,blue -Gblack -Se0.02/0.65/12 -A10p+ea+n10 
gmt end

In [ ]:
Image('demo.png', width=500)

### Python example

In [ ]:
dem='../6_geodesy/dem_dsamp.grd'
region = pygmt.grdinfo(dem, spacing="-")[2:-1]
fig = pygmt.Figure()
fig.basemap(projection="M6", region=region, frame="a1f0.5")
fig.grdimage(dem, shading="+nt.3", cmap="geo")
fig.coast(borders="a/0.5p,black,-.-", water="lightblue", resolution="f")
tmpe = pygmt.grdsample('../6_geodesy/dE.grd', spacing=0.1)
tmpn = pygmt.grdsample('../6_geodesy/dN.grd', spacing=0.1)
tmpe_lld = pygmt.grd2xyz(tmpe)
tmpn_lld = pygmt.grd2xyz(tmpn)
data = pd.concat([tmpn_lld[['x', 'y']], tmpe_lld['z'], tmpn_lld['z'], pd.DataFrame(np.zeros((len(tmpn_lld['z']), 3)))], axis=1)
fig.velo(data, pen="0.1p,black", spec="e0.02/0.65/10", vector="10p+eA+n10")
fig.plot('../6_geodesy/new_faults.gmt', pen="0.5p,red")
fig.velo('../6_geodesy/GPS.dat', pen="1p,blue", color="black", spec="e0.02/0.65/12", vector="10p+ea+n10")
fig.show()

## Stay involved!

We'd love for you to get more involved in PyGMT! Some ideas are to [post on the PyGMT forum Q&A](https://forum.generic-mapping-tools.org/c/questions/pygmt-q-a/11), help [contribute to our documentation](https://github.com/GenericMappingTools/pygmt/issues?q=is%3Aopen+is%3Aissue+label%3Adocumentation), or [add some features](https://github.com/GenericMappingTools/pygmt/issues?q=is%3Aopen+is%3Aissue+label%3A%22feature+request%22)! As always, get in touch with any questions!